# 11. Attention Mechanisms and Transformers

- ReLU, BatchNorm과 같은 다양한 기법들이 등장하였지만
- 필수적인 Design 요소는 지난 30년간 크게 바뀐 것 없었고 컴퓨터 성능과 방대한 데이터가 DNN의 성장 동인이 되어 왔다.
- 하지만 Transformer 등의 새로운 아키텍쳐가 등장하면서 DNN 분야에 새로운 성장을 이루어내고 있다.
- 대부분의 NLP의 모델이 Transformer를 기반으로 하고 있으며 새로운 Task를 다룰 때 첫번째 고려대상으로 인식되고 있다.
- NLP뿐아니라 Vision 영역에서 ViT (Vision Transformer) 등이 주목받고, Speech Recognition, Reinforcement Learning, Graph Neural Network 등 많은 다른 분야에 사용되고 있다.

## Idea behind transformers

- Seq2Seq (e.g. machine translation) 모델의 성능을 개선하기 위해 최초로 개발되었다. 
- Decoder가 입력 Sequence에 따라 특정 부분에 더욱 가중치를 둘 수 있도록 하기 위한것으로써 설계.
  - Encoder는 입력 Sequence와 같은 크기의 Representation을 생성하고 Decoder는 이 Repr.의 Weighted Sum으로 이루어진 Context Vector를 입력으로 받아 새로운 Seqeunce를 생성
  - 여기서 Weight은 Sequence의 각 요소에 얼마 집중할 것인지를 나타냄
  - 그리고 이 Weight를 할당하는 과정이 미분 가능하며 다른 paramter와 같이 학습 가능함.
- 기존의 Seq2Seq SOTA를 갈아치우며 RNN 기반 Seq2Seq의 성공적인 개선으로써 도입됨.
- 게다가 일부 Seq2Seq에서 대응 단어에 높은 Weight을 할당하는 등과 같이 매우 직관적인 해설(Interpretability)을 제공

## Dominance of Transformers

- 단순히 Seq2Seq에서 Salient Input을 선별하는 것에 그치지 않고 Vaswani 등에 의해 온전히 Transformer를 기반으로한 Machine Translation 방식이 제안('17)되었고 당시 최고 성능의 모델을 넘어서면서 주목받기 시작
- NLP에 Transformer가 새로운 기준으로 수용되고 이러한 모델들이 기존의 SOTA를 갈아치우기 시작함.
- NLP의 지배적인 설계 접근 방식으로써...
  - 다양한 광범위한 Corpora (Text Dataset)을 이용..
  - Self-supervised 방식의 Pre-training
  - Downstream Task 데이터를 이용한 Fine-Tuning
- Pretraining Paradigm을 적용한 이후 기존의 Architecture 기반의 모델과의 Gap이 더욱 극대화됨..



## Queries, Keys and Values

- 많은 DNN 모델들이 학습 및 추론의 효율성을 위해 fixed input size에 의존함.
- 이러한 fixed input size는 다양한 길이의 Text를 다루는 NLP에서 문제점으로 작용.
- 이러한 문제점의 해결방안으로써 Recurrent Architecture를 이용하여 입력을 순차적인 방식으로 처리하거나..
- 특수하게 설계된 Conv Kernel을 이용하는 등의 접근이 있어왔다...
- 하지만 Sequence의 길이가 길어질 수록 아래와 같은 다양한 문제점들이 발생
  - 낮은 연산 효율성
  - 과도한 메모리 사용
  - 멀리 떨어진 정보간의 의존성

### Database와의 비유
- Database는 Key-Value pair로 구성된 데이터 집합으로 Query를 통해 특정 정보를 요청할 수 있다.
- 이때 Query는 Database의 Size의 변화와 상관 없이 Design 될 수 있으며 이러한 변화에 대해 효율성의 저하가 없다.
- 또 한편으로는 Database의 정보의 복잡도와 크기에 어떠한 제약이 없으며, 이러한 복잡도의 증가가 Query의 효율성에 영향을 주지 않는다.
- 데이터베이스에 포함된 정보에 따라 같은 Query에 대해 다른 결과가 나올 수 있다.
- Attention Mechanism은 이러한 Database의 Query의 특징에 영감을 받아 고안되었다.
- 이를 통해 주어진 Query에 따라 Model이 입력의 특정 영역에 보다 집중할 수 있도록 설계되어졌다. (이는 Database가 주어진 Query에 대해 특정 K-V pair에 집중하는 것과 유사)

### Attention
- Attention Weight
  
  > a_i =  q_i@k_i
  
  - 각 attention weight는 convex cone
  - 각 attention의 합은 이러한 convex들의 조합으로 나타나며 (울퉁불퉁한 wave) 전체 합은 1이됨. (softmax)
  - 전체 element 중 한 weight가 1이면 앞의 특징들에 의해 나머지 elem. weight은 0이되며 이는 Databse의 query와 유사
  - 만약 모든 elem.의 weight가 uniform하게 같으면 결국 average pooling과 같음
  - (참고) 대표적인 특징에 있어서는 동일하나 상세 설계 등에 있어 다양. RL에 의해 학습되는 Non-differentiable Attention 도 있다. 

In [ ]:
from torch import nn, Tensor


class DotProductAttention(nn.Module):
    
    def __init__(self, dropout:float, num_heads:int=None) -> None:
        super().__init__()
        self.dropout = dropout
        self.num_heads = num_heads
        
    def forward(self, queries: Tensor, keys: Tensor, values: Tensor, valid_lens:Tensor=None, window_mask:Tensor=None):
        pass